## Impot Libraries

In [107]:
import pandas as pd
from pyomo.environ import *

In [108]:
# --- 1. Einlesen der Daten ---
students = pd.read_excel("studierende.xlsx", sheet_name="Alle Studierende")
unis = pd.read_excel("unis.xlsx", sheet_name="Alle Universitäten")

# Studenten

In [109]:
students.head()

,Vorname,Nachname,Level,Matrikelnummer,Dauer,Semesterwahl,BesondereChance:Behinderung,BesondereChance:Kind,Note,Motivation,Sprache,Lebenslauf,Formalien,ECTS,Gasthochschule 1. kurz,Gasthochschule 2. kurz,Gasthochschule 3. kurz,Gasthochschule 4. kurz,Gasthochschule 5. kurz,Programm
0,Vorname-01,Nachname-01,Master,1000001,1,WISE,False,False,1.35,1,C2,1,1,50,Uni-02,Uni-03,Uni-100,NaN,NaN,Programm-01
1,Vorname-02,Nachname-02,Bachelor,1000002,1,WISE,False,False,1.74,2,B2,2,1,30,Uni-08,Uni-100,Uni-83,NaN,NaN,Programm-02
2,Vorname-03,Nachname-03,Bachelor,1000003,1,WISE,False,False,1.63,1,C1,1,1,45,Uni-26,Uni-98,Uni-106,Uni-65,NaN,Programm-03
3,Vorname-04,Nachname-04,Master,1000004,1,SOSE,False,False,1.40,3,B1,3,2,67,Uni-09,Uni-24,NaN,NaN,NaN,Programm-04
4,Vorname-05,Nachname-05,Master,1000005,1,WISE,False,False,1.16,2,C2,1,1,44,Uni-09,NaN,NaN,NaN,NaN,Programm-05


In [110]:
students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Vorname                      209 non-null    object 
 1   Nachname                     209 non-null    object 
 2   Level                        209 non-null    object 
 3   Matrikelnummer               209 non-null    int64  
 4   Dauer                        209 non-null    int64  
 5   Semesterwahl                 209 non-null    object 
 6   BesondereChance:Behinderung  209 non-null    bool   
 7   BesondereChance:Kind         209 non-null    bool   
 8   Note                         209 non-null    float64
 9   Motivation                   209 non-null    int64  
 10  Sprache                      209 non-null    object 
 11  Lebenslauf                   209 non-null    int64  
 12  Formalien                    209 non-null    int64  
 13  ECTS                

## Gewichtung der Kriterien zur Bewertung

In [113]:
# Gewichtung der Spalten
weights = {
    'Note': 0.6,                   # 1.0 - 5.0
    'Motivation': 0.2,             # 1, 2, 3
    'Sprache': 0.1,                # C2, C1, B2, B1, A2, A1
    'Lebenslauf': 0.05,             # 1, 2, 3
    'Formalien': 0.05,              # 1, 2, 3
}


## Hilfsfunktionen für Bewertung der Kriterien

In [114]:
# Hilfsfunktionen zur Normierung der Kriterien
def normalize_note(note):
    # Beste Note = 1.0, schlechteste = 5.0
    return (5.0 - note) / 4.0

def normalize_motivation(motivation):
    # 1 = beste Motivation, 3 = schlechteste
    return (3 - motivation) / 2.0

def normalize_sprache(sprache):
    # Sprachlevel Mapping
    mapping = {'C2': 1.0, 'C1': 0.8, 'B2': 0.6, 'B1': 0.4, 'A2': 0.2, 'A1': 0.0}
    return mapping.get(sprache, 0.0)

def normalize_lebenslauf(lebenslauf):
    # 1 = sehr gut, 3 = schlecht
    return (3 - lebenslauf) / 2.0

def normalize_formalien(formalien):
    # 1 = sehr gut, 3 = schlecht
    return (3 - formalien) / 2.0

## Anwendung der Kriterien und Gewichtungen

In [115]:
# Anwenden der Gewichtung auf alle Studierenden
def calculate_score(row):
    score = 0
    score += weights['Note'] * normalize_note(row['Note'])
    score += weights['Motivation'] * normalize_motivation(row['Motivation'])
    score += weights['Sprache'] * normalize_sprache(row['Sprache'])
    score += weights['Lebenslauf'] * normalize_lebenslauf(row['Lebenslauf'])
    score += weights['Formalien'] * normalize_formalien(row['Formalien'])
    return score

# Score berechnen
students['Score'] = students.apply(calculate_score, axis=1)

## Output der Scores je Student

In [116]:
print(students[['Matrikelnummer', 'Score']])

print("Min Score:", students['Score'].min())
print("Max Score:", students['Score'].max())
print("Average Score:", students['Score'].mean())
print("Median Score:", students['Score'].median())

     Matrikelnummer   Score
0           1000001  0.9475
1           1000002  0.7240
2           1000003  0.8855
3           1000004  0.6050
4           1000005  0.8760
..              ...     ...
204         1000205  0.9205
205         1000206  0.8500
206         1000207  0.7120
207         1000208  0.9090
208         1000209  0.9285

[209 rows x 2 columns]
Min Score: 0.15050000000000002
Max Score: 0.9475000000000001
Average Score: 0.6702727272727274
Median Score: 0.6955000000000001


## Besondere Chancen & Malus

In [117]:
# Score um 0.6 erhöhen, falls BesondereChance:Behinderung oder BesondereChance:Kind True ist
students.loc[
    (students['BesondereChance:Behinderung'] == True) | (students['BesondereChance:Kind'] == True),
    'Score'
] += 0.6

# Score um 0.2 verringern, falls Abschluss "Bachelor" und ECTS < 60
students.loc[
    (students['Level'] == 'Bachelor') & (students['ECTS'] < 60),
    'Score'
] -= 0.2

In [118]:
print(students[['Matrikelnummer', 'Score']])

print("Min Score:", students['Score'].min())
print("Max Score:", students['Score'].max())
print("Average Score:", students['Score'].mean())
print("Median Score:", students['Score'].median())

     Matrikelnummer   Score
0           1000001  0.9475
1           1000002  0.5240
2           1000003  0.6855
3           1000004  0.6050
4           1000005  0.8760
..              ...     ...
204         1000205  0.9205
205         1000206  0.8500
206         1000207  0.5120
207         1000208  0.7090
208         1000209  0.9285

[209 rows x 2 columns]
Min Score: 0.15050000000000002
Max Score: 1.1975000000000002
Average Score: 0.6291244019138758
Median Score: 0.6580000000000001


# Universitäten

In [119]:
unis.head()

,ArbeitsnameUni,Status,Liste,GleicheAufteilungWISESOSE,MaxBachelor,MaxMaster,MaxBeide,Programm-03,Programm-08,Programm-02,...,Programm-09,Programm-16,Programm-11,Programm-17,Programm-13,Programm-18,Programm-10,Programm-05,Programm-01,Programm-04
0,Uni-01,Aktiv,BWL,False,2,0,0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Uni-01,Aktiv,SozÖk,False,0,2,0,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,Uni-01,Aktiv,Politik,False,0,0,2,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,Uni-02,Aktiv,Journa,False,0,2,0,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,Uni-03,Aktiv,Politik,False,0,0,1,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [120]:
unis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ArbeitsnameUni             168 non-null    object
 1   Status                     168 non-null    object
 2   Liste                      168 non-null    object
 3   GleicheAufteilungWISESOSE  168 non-null    bool  
 4   MaxBachelor                168 non-null    int64 
 5   MaxMaster                  168 non-null    int64 
 6   MaxBeide                   168 non-null    int64 
 7   Programm-03                168 non-null    bool  
 8   Programm-08                168 non-null    bool  
 9   Programm-02                168 non-null    bool  
 10  Programm-06                168 non-null    bool  
 11  Programm-07                168 non-null    bool  
 12  Programm-15                168 non-null    bool  
 13  Programm-12                168 non-null    bool  
 14  Programm-1

## Universitäten mit aktuellem Aufnahmestopp entfernen

In [ ]:
unis = unis[unis['Status'] != 'Pausiert']